In [ ]:
#IMPORT CONFIG FROM FILE .CONFIG
import configparser
import os
import re
config_file_name = 'price_monitoring.config'
#cwd = os.path.dirname(sys.argv[0]) #working on linux server or windows python script
cwd = os.getcwd() #working on windows during python notebook execution
cfg = configparser.ConfigParser()
ini_config_path = os.path.join(cwd,config_file_name)
cfg.read(ini_config_path)
bot_token = cfg['telegram_config']['bot_token']

In [ ]:
# import telebot

# bot = telebot.TeleBot(bot_token)

# @bot.message_handler(commands=['start', 'help'])
# def send_welcome(message):
# 	bot.reply_to(message, "Howdy, how are you doing?")

# @bot.message_handler(func=lambda message: True)
# def echo_all(message):
# 	chat_id = message.chat.id
# 	print(message)
# 	msg_reply = 'chat_id: ' + str(chat_id) + ' message: ' + message.text
# 	bot.reply_to(message, msg_reply)

# bot.infinity_polling()

In [ ]:
def get_url(text):
    try:
        url = re.search(r'(https?://[^\s]+)', text).group(1)
    except:
        url = None
    return url

def validate_number(text):
    try:
        text = re.search(r'(^\d+(,\d+)*$)', text).group(1)
    except:
        text = None
    return text

In [ ]:
import telebot
from telebot import types

bot = telebot.TeleBot(bot_token)

user_dict = {}

class User:
    def __init__(self, chat_id):
        self.chat_id = chat_id
        self.url = None
        self.target_price = None

# Handle '/help' command
@bot.message_handler(commands=['help'])
def send_welcome(message):
    msg = bot.reply_to(message, """\
Ola, eu sou o bot de monitoramento de precos.
Envie o comando /start para iniciar.
""")

# Handle '/start' command
@bot.message_handler(commands=['start'])
def process_start_menu(message):
    msg_reply = "Ola, eu sou o bot de monitoramento de precos. O que deseja fazer?"
    try:
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Cadastrar', 'Consultar', 'Sair')
        msg = bot.reply_to(message, msg_reply, reply_markup=markup)
        bot.register_next_step_handler(msg, process_start_menu_handler)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_start_menu_handler(message):
    try:
        answer = message.text
        if answer == u'Cadastrar':
            msg = bot.reply_to(message, 'Envie o link do produto que deseja monitorar')
            bot.register_next_step_handler(msg, process_url_step)
        elif answer == u'Consultar':
            #aqui tem que trocar mensagem e a funcao de consulta
            msg = bot.reply_to(message, 'Envie o link do produto que deseja monitorar')
            bot.register_next_step_handler(msg, process_url_step)
        elif answer == u'Sair':
            bot.reply_to(message, 'Obrigado, ate mais')
        else:
            raise Exception("Mensagem desconhecida")
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')


def process_url_step(message):
    try:
        chat_id = message.chat.id
        user = User(chat_id)
        user_dict[chat_id] = user
        url = get_url(message.text)
        
        if url is not None:
            user.url = url
            msg = bot.reply_to(message, 'Informe o preco alvo do produto:')
            bot.register_next_step_handler(msg, process_target_price_step)
        else:
            msg = bot.reply_to(message, 'O link nao e valido. Informe novamente o link do produto:')
            bot.register_next_step_handler(msg, process_url_step)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')

def process_target_price_step(message):
    try:
        chat_id = message.chat.id
        target_price = message.text
        if validate_number(str(target_price)) is None:
            msg = bot.reply_to(message, 'O preco deve ser um numero. Utilize virgula para separar casas decimais.\n Informar novamente o preco alvo do produto:')
            bot.register_next_step_handler(msg, process_target_price_step)
            return
        user = user_dict[chat_id]
        user.target_price = target_price
        msg_reply = 'ID: ' + str(chat_id) + '\nURL: ' + user.url + '\nPreco alvo: ' + str(user.target_price)
        bot.send_message(chat_id, msg_reply)
    except Exception as e:
        bot.reply_to(message, 'oooops, algo nao esta certo, tente novamente')    

# Enable saving next step handlers to file "./.handlers-saves/step.save".
# Delay=2 means that after any change in next step handlers (e.g. calling register_next_step_handler())
# saving will hapen after delay 2 seconds.
bot.enable_save_next_step_handlers(delay=2)

# Load next_step_handlers from save file (default "./.handlers-saves/step.save")
# WARNING It will work only if enable_save_next_step_handlers was called!
bot.load_next_step_handlers()

bot.infinity_polling()

In [ ]:

def process_name_step(message):
    try:
        chat_id = message.chat.id
        name = message.text
        user = User(name)
        user_dict[chat_id] = user
        msg = bot.reply_to(message, 'How old are you?')
        bot.register_next_step_handler(msg, process_age_step)
    except Exception as e:
        bot.reply_to(message, 'oooops')


def process_age_step(message):
    try:
        chat_id = message.chat.id
        age = message.text
        if not age.isdigit():
            msg = bot.reply_to(message, 'Age should be a number. How old are you?')
            bot.register_next_step_handler(msg, process_age_step)
            return
        user = user_dict[chat_id]
        user.age = age
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Male', 'Female')
        msg = bot.reply_to(message, 'What is your gender', reply_markup=markup)
        bot.register_next_step_handler(msg, process_sex_step)
    except Exception as e:
        bot.reply_to(message, 'oooops')


def process_sex_step(message):
    try:
        chat_id = message.chat.id
        sex = message.text
        user = user_dict[chat_id]
        if (sex == u'Male') or (sex == u'Female'):
            user.sex = sex
        else:
            raise Exception("Unknown sex")
        bot.send_message(chat_id, 'Nice to meet you ' + user.name + '\n Age:' + str(user.age) + '\n Sex:' + user.sex)
    except Exception as e:
        bot.reply_to(message, 'oooops')


In [ ]:
import telebot

bot = telebot.TeleBot(bot_token)

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
	bot.reply_to(message, message.text)

bot.infinity_polling()